# Contextualized Word Embedding
- Word Embedding처럼 단어를 저차원 벡터 공간에 표현하되, **각 단어가 문맥 내에서 가지는 의미를 반영하는** 문맥 기반 단어 표현 방식이다.
  
## Word Embedding과의 차이

- 기존 Word Embedding 방식은 **동음이의어**(같은 발음에 여러 뜻을 가진 단어)가 **같은 벡터**로 표현되기 때문에, 문장의 의미(내용)이  제대로 표현되지 못할 수 있다.  
    - 예: "밤에 만나자."와 "어제 먹은 밤 정말 맛있었다."라는 두 문장에서 '밤'은 서로 다른 의미이지만, word2vec과 같은 Word Embedding 방식에서는 동일한 벡터로 표현된다.
- **Contextualized Word Embedding**은 동일한 발음의 단어라도 **문맥에 따라 각기 다른 벡터를 생성하여** 단어의 의미를 정확히 표현하는 것을 목표로 한다.  
    - 예: Contextualized Word Embedding은 **동의 이의어의 단어라도 그 문장에서 어떤 의미로 사용되는지에 맞게 다르게 벡터화**한다. 따라서 위의 예의 두 문장에서 사용된 '밤'은 서로 다른 벡터로 표현된다.
 
## Contextualized Word Embedding 구현 방식
- 딥러닝 모델로 구현한다.
    - Word2vec이 하나의 hidden layer로 구성되는 것과는 달리, 많은 layer들을 연결해서 deep한 네트워크 모델로 정의한다.
- Contextualized Word Representation은 문장을 입력 받아 그 문장 내의 각 단어에 대한 표현을 추정한다.
    - 문장을 구성하는 단어들을 기존 word embedding 벡터값 입력받은 뒤, neural network의 layer들을 거치면서 문장(문맥)내에서 어떤 의미로 사용되었는지를 표현하는 vector를 생성한다.
- 예전에는 RNN계열의 딥러닝 모델들을 사용했으나,  최근에는 [Attention Is All You Need](https://arxiv.org/pdf/1706.03762) 논문에서 제안한 Transformer 계열의 모델을 주로 사용한다.
- **대표적 모델**
    - ELMo (Embeddings from Language Models)
    - BERT (Bidirectional Encoder Representations from Transformers)
    - GPT (Generative Pre-trained Transformer)
    - RoBERTa (A Robustly Optimized BERT Pretraining Approach)
- 대부분 대규모 Dataset에서 사전 학습(pre-training)모델을 자신의 작업에 맞춰 미세 조정(fine-tuning)하여 사용한다.

> - **동음이의어**
>     - 같은 발음은 같은데 뜻이 다른 단어를 **동음이의어** 라고 한다.
>     - 예로 먹는 "밤" 과 해가 떨어진 시간대를 나타내는 "밤", 먹는 "배"와 타는 "배"와 사람의 "배"
> - **동음다의어**
>     - 하나의 단어가 여러개의 관련된 뜻을 가진 것을 **동음다의어**라고 한다.
>     - 예) "머리를 끄덕이다.", "머리가 좋다." , "머리가 길다"  셋다 관련되 있지만 다른 뜻으로 쓰인다.
>         - [네이버사전 머리뜻](https://ko.dict.naver.com/#/entry/koko/a90afd48126045a09bc9b6c634b7ff54)을 보면 머리에서 파생된 의미가 11가지가 나와 있다. (중심의미로 부터 파생된 파생의미들.)


# Recurrent Neural Network (RNN)

- Sequential data 의 특성을 추출하는데 좋은 성능을 보이는 Recurrent Layer를 Feature Extractor로 사용하는 딥러닝 모델.
- 시계열 데이터, 자연어처리, 음성데이터 처리에 많이 사용된다.

# Sequential 데이터란

- 데이터의 **순서가 있고 그 순서 정보가 중요한 데이터셋**
    - 순서가 달라질 경우 의미가 바뀌거나 손상되는 데이터.
    - 처리시 **순서의 흐름이(Context:문맥) 중요한 데이터**
        - 현재 순서의 값을 처리할 때 이전 순서까지의 처리 결과들을 모두 같이 고려해야 하는 경우.
- 예
    - 자연어 텍스트
    - 일정한 주기로 샘플링된 영상, 음성
    - 시계열(time series) 데이터
> - Time series
>    - Sequential 데이터 중 데이터의 순서에 뿐 아니라 해당 **데이터가 발생한 시점 정보가 중요한 데이터셋을** 말한다.

# RNN (Recurrent Neural Network) 구조

## 개요
- RNN은 Feature Extractor로 Recurrent Layer을 사용하는 딥러닝 모델(네트워크)을 말한다.
- Recurrent Layer는 sequential 데이터 처리에 좋은 성능을 낸다.

![rnn_outline](figures/rnn/RNN_outline.png)

## Recurrent Layer 구조
- Sequential 데이터는 순서에 맞춰 처리하는 것이 중요하다. 그러나 DNN과, CNN에서 사용되는 Fully Connected Layer나 Convolutional Layer는 순서를 고려하지 않고 특성 추출 한다.
- RNN은 순서대 입력되는 데이터를 반복 처리하는  **Recurrent Layer**를 이용해 **순서를 가만해서 Feature vector를 추출하고** 그 Feature vector를 Estimator Layer에 전달해 추론한다.
- **순서를 감안한다는 것**은 이전 순서의 내용을 기억한다는 것을 말한다. 이것을 memory system이라고 한다.

### 메모리 시스템(Memory system)
- **이전 시점까지의 특성들을 이용해 현재 시점의 특성을 추출한다.**
    - Sequential Data는 단순히 순서대로 처리하는 것 뿐 만 그 내용들을 **기억(memory)** 하고 **참고** 해서 현재 순서의 내용을 처리해야 한다.
    - 메모리 시스템이란 **"이전 단계의 처리결과를 기억하고 그것을 현재 단계 처리에 사용하는"** 것으로 Recurrent layer의 핵심 로직이다.

> Sequential 데이터 처리에서 순서대로 처리할 때 각각의 단계(순서)를 **time step** 이라고 한다.
 
![memory system](figures/rnn/01_memory_system.png)

- 예를 들어 4일간의 주가 변화로 5일째 주가를 예측하려면 입력받은 4일간의 주가 정보를 순서대로 기억하고 있어야 한다.
- Fully Connected Layer나 Convolution Layer의 출력은 이전 Data에 대한 처리와 상관없이 현재 시점의 데이터만을 기준으로 특성(feature)을 추출한다.

### Simple RNN 구조
![rnn arch](figures/rnn/02_simplernn.png)


![rnn_arch2](figures/rnn/03_simplernn_2.png)

- Recurrent Layer는 Linear layer 구조에 **재귀 순환(반복)의 로직**이 들어간 것으로 이해할 수 있다.
- Layer의 한 step은 입력으로 **현재 시점(time step)의 입력 데이터($X_{t}$)** 와 **이전 시점까지의 처리결과($h_{t-1}$)** 를 같이 받는다.  (t: time step)
- $X_{t}$와 $h_{t-1}$ 에 각각의 weight($W_{xh}\,, W_{hh}$)를 이용해 가중합을 구하고 그 둘의 합계를 activation(tanh) 함수에 넣어 비선형성을 추가한 결과가 현재 time step의 출력값이된다. **이 출력값이 현재 time step의 feature(특성)값** 이다.
    - 이 출력값은 현재 time step의 feature 이자 **다음 time step의 feature 계산의 hidden state 입력값으로 사용된다.**
    - $X_{t}$(현재 time step의 입력)와 $h_{t-1}$(hidden state) 에는 각각 다른 weight($W_{xh}$ , $W_{hh}$)들이 적용된다.
- 매 time step마다 동일한 weight $W_{xh}$와 $W_{hh}$ 가 사용된다.
    - time step 마다 새로운 weight들이 적용되는 것이 아니다.

> #### Hidden state
> - 이전 step까지의 처리결과를 말한다..
> - **메모리 시스템의 메모리(기억)** 에 해당한다.
> - 현재 timestep의 feature를 추출할 때 현재 시점의 입력과 함께 recurrent layer의 입력으로 들어간다.


## NLP 문제 유형과 RNN 구조

|유형|구조1|구조2|구현서비스(App)|
|:-|:-|:-:|:-|
|**Many to One**|입력: Sequence Vector(여러개의 Vector)<br>출력: 1개 Vector|![mto](figures/rnn/04_many_to_one.png)|$\scriptscriptstyle\blacksquare$ Text classification(감성 분석)|
|**One to Many**|입력: 1개 Vector<br>출력: Sequence Vector|![otm](figures/rnn/04_one_to_many.png)|$\scriptscriptstyle\blacksquare$ Image captioning<br>$\scriptscriptstyle\blacksquare$ 문장생성|
|**Many to Many**|입력: Sequence Vector<br>출력: Sequence Vector|![mtm1](figures/rnn/04_many_to_many.png)|$\scriptscriptstyle\blacksquare$ 개체명인식<br>$\scriptscriptstyle\blacksquare$ 품사태깅|
|**Seq2Seq**|many to one과 one to many 를 연결한 구조.<br>Encoder, Decoder 구조<br>Encoder는 특성을 추출하고 <br>Decoder는 Encoder가 추출한 특성을 이용해 결과를 생성.|![seq2seq](figures/rnn/04_seq2seq.png)|$\scriptscriptstyle\blacksquare$ Machine Translation<br>$\scriptscriptstyle\blacksquare$ Chatbot|


- **텍스트 분류(Text classification)**
   - 입력받은 문장을 분류하는 문제로 대표적으로 감성분석(sementic analysis)가 있다.
   - 감성분석
       -  입력받은 텍스트가 어떤 감정의 글인지를 분류한다. 일반적으로 긍정, 중립, 부정적 인지를 분류한다.
- **Image captioning**
   - 입력받은 이미지를 설명하는 문장을 생성하는 문제.
- **개체명인식(Named Entity Recognition)**
   - 문장의 각 단어가 어떤 종류(의미) 인지를 찾는 문제
   - 미국에 사는 톰은 스무살입니다. ==> 미국: 위치, 톰: 이름, 스므살: 나이
- **품사태깅(Pos tagging)**
   - 문장의 각 토큰의 품사를 찾는 문제
   - 미국에 사는 톰은 스무살입니다. ==> 미국: 대명사, 예: 조사,  톰: 대명사, 은: 조사, 스무: 명사, 살: 명사, 이다: 동사
- **Chatbot**
   - 입력받은 문장에 대한 답을 하는 시스템.
   - Encoder는 질문을 받아 처리하고 Decoder는 답변을 생성하는 seq2seq 구조를 사용한다.
- **Machine translation**
   - 번역 시스템
   - Encoder는 번역 대상문장을 입력받아 처리하고 Decoder는 번역 문장을 생성하는 seq2seq 구조를 사용한다.

## Recurrent Layer 구조들

### Bidirectional RNN
- 같은 정보를 정방향과 역방향 두 방향으로 주입해 정확도를 높인다.
- Non-Auto Regressive 모델의 경우 bidirectional RNN 사용이 권장된다.

> - **Auto Regressive 모델**
>     - 이전 time step에 대한 출력결과를 다음 time step의 입력으로 사용하는 구조.
>         - Text 생성 모델이 대표적인 Auto Regressive 모델이다.
>     - 이전 출력의 결과에 의존하기 때문에 bidirectional RNN을 사용할 수 없다.
>     - Non Auto Regressive 모델은 현재의 상태가 앞/뒤 상태에 따라 정해지는 경우로 보통 앞/뒤의 상태를 모두 참조할 경우 성능이 올라간다. 그래서 단방향 RNN보다 bidirectional RNN의 사용이 권장된다.

![bidirectional](figures/rnn/08_bidirectional.png)

### Stacking(Multi Layer) RNN

- Recurrent Layer를 쌓아 모델의 용량을 늘려 표현능력(Represenational capacity)를 증가시킨다.
- 여러층을 쌓은 경우 먼저 쌓은 layer의 모든 time step별 출력이 다음 Layer의 입력 데이터로 사용된다.
- Layer을 쌓으면 표현능력은 증가하는 대신 계산 비용이 많이들고 **과대적합(Overfitting)이** 발생할 수 있다.
    - 과대적합을 막기 위해 드롭아웃 레이어를 추가할 수있다.
    - Recurrent Layer에서 Dropout layer는 매 time step 마다 적용한다.

![stacking rnn](figures/rnn/08_multi_layer_rnn.png)

## Pytorch RNN Layer

- [nn.RNN](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html)
    - **파라미터**
        - input_size: 입력 데이터의 크기 (feature수)
        - hidden_size: Layer의 Hidden size
        - num_layers: 몇 층으로 Layer을 쌓을지 개수
        - nonlinearity: 활성함수(Activation Function) 지정. 문자열로 'tanh' (Default) 또는 'relu' 둘 중 하나 지정.
        - batch_first: True - (batch, seqence len, ..) False - (sequence len, batch, ..). Default: False
        - dropout: Dropout rate 비율
        - bidirectional:  양방향 적용 여부. Default: False
   

  
### RNN Layer의 input / output tensor 의 shape

### 추론시 Input
- 입력으로 두개를 받는다.
    - Input_Data, Hidden_state
- **Input_data의 shape**
    - (Sequence_legnth, batch_size, feature_shape)
        - pytorch는 입력으로 batch보다 sequence length가 먼저 온다.
        - `batch_first=True`로 설정하면 (**batch_size**, seq_len, feature_shape) 순이 된다.
        - ex)  주가 데이터
            - feature: 시가, 종가, 최고가, 최저가 (4개)
            - sequence: 100일치( 100개의 feature가 하나의 입력이 된다.)
            - batch size: 30 (100일치 데이터 30개)
            - (100, 30, 4)
    ![input tensor shape](figures/rnn/07_input_shape.png)
- **Hidden state의 shape**
    - 시작(초기) hidden state로 입력하지 않으면 0이 들어간다.
    - shape은 아래 hidden state 설명 참조

### Output
- **(output_data, hidden_state)**
    - output_data과 hidden state를 tuple로 묶어서 반환한다.
- **Output_data**:
    - 매 time step의 출력결과
    - Many to Many일 경우 이것을 출력 Feature들로 사용한다.
- **hidden_state**
    - 마지막 time step의 출력결과
    - Many to one의 경우 이것을 출력 Feature로 사용한다.
  
#### Output shape
- **(Sequence length, batch_size, hidden_size * D)**
    - D: 양방향(bidirectional) 이면 2 아니면 1
    - batch_first=True 로 설정하면 (**batch_size**, seq_len, hidden_size * D) 순이 된다.
    - ex)
        - RNN Layer의 hidden size가 256 인 경우. (sequence length: 100, batch size: 30, bidirection=False)
        -  (100, 30, 256)
![hidden state shape](figures/rnn/07_hidden_state_shape.png)

#### Hidden state
- 마지막 time step의 출력결과
    - **(D * layer수, batch_size, hidden_size)**
    -  D: 양방향(bidirectional) 이면 2 아니면 1
    -  layer수: multi layer일 경우 layer stack 수
        -  각 Layer 별 마지막 time step의 결과들이 묶여서 hidden state 가 된다.

In [1]:
import torch
import torch.nn as nn

In [2]:
# 입력데이터 - 3차원 batch_first = False
# seq_length : data를 구성하는 timestep의 개수 -> 개별 문서의 단어수.
# batch_size : 입력 데이터의 개수 -> 한번에 입력되는 문서의 개수.
# feature : 각 timestep을 구성하는 feature의 개수. -> 단어의 임베딩 차원수.

input_data = torch.randn((30, 100, 4)) # batch_first = False (seq_len, batch_size, feature)
# 개별 단어 : 4개
# 한개의 문장(문서) : 30개 단어로 구성
# 총 100개의 문장

In [ ]:
# layer수 : 1, 단방향
rnn1 = nn.RNN(
    input_size = 4, # 개별 timestep의 feature 수
    hidden_size = 256, # 개별 timestep별로 추출할 feature의 개수. Unit/Node의 개수
    num_layers = 1, # 멀티레이어 RNN - > 쌓을 레이어의 개수
    # batch_first = True # (batch, seq_length, feature)
    # nonlinearity = "relu" # 활성함수. defalut : "tanh"
)

# output = rnn1(input_data) # (seq_length : 30, batch_size : 100, feature : 4)
# print(type(output)) 
# (output_data : 모든 timestep의 hiddenstata, hidden_state : 마지막 timestep의 hiddenstate)
out1, hidden1 = rnn1(input_data)
print(out1.shape) # 모든 timestep의 결과
print(hidden1.shape) # 마지막 timestep의 결과


torch.Size([30, 100, 256])
torch.Size([1, 100, 256])


In [ ]:
# layer수 : 1, 양방향
rnn2 = nn.RNN(
    input_size = 4, 
    hidden_size = 256, 
    num_layers = 1,
    bidirectional = True, # 양방향 RNN 여부 (default : False)
)

out2, hidden2 = rnn2(input_data)
print(out2.shape) # [30, 100, 512 순/역방향 결과를 concat]
print(hidden2.shape) # [2:순/역방향, 100, 256]

torch.Size([30, 100, 512])
torch.Size([2, 100, 256])


In [ ]:
# layer수 : 3, 단방향
rnn3 = nn.RNN(
    input_size = 4, 
    hidden_size = 256, 
    num_layers = 3,   
)

out3, hidden3 = rnn3(input_data)
print(out3.shape) # 모든 timestep의 (마지막 layer가 출력한) hiddenstate들.
print(hidden3.shape) # [3 : 레이어수, 100, 256] -> layer별 마지막 timestep의 hidden state들을 모아서 반환.

torch.Size([30, 100, 256])
torch.Size([3, 100, 256])


In [ ]:
# layer수 : 3, 양방향
rnn4 = nn.RNN(
    input_size = 4, 
    hidden_size = 256, 
    num_layers = 3,
    bidirectional = True, # 양방향 RNN 여부 (default : False)
)

out4, hidden4 = rnn4(input_data)
print(out4.shape) # [30, 100, 512 순/역방향 결과를 concat]
print(hidden4.shape) # [6:순/역방향(2) * 3, 100, 256]

torch.Size([30, 100, 512])
torch.Size([6, 100, 256])


# RNN의 Back Propagation 
- **BPTT(Back Propagation Through Time)** 이라고 한다.
- RNN time step 별로 순환 반복는 hidden state의 흐름의 역방향으로 전파되는 gradient 들이 chain rule에 의해 곱해지면서 weight가 업데이트 된다.
    - 각 time step 마다 사용되는 weight는 같기 때문에 1 step에서 weight는 여러번(time step수만큼)에 걸쳐 업데이트 된다.

## RNN(Simple RNN)의 문제 
- 입력 데이터의 sequence가 길수록 Gradient Vanishing이 발생해 초기 Sequence에 대한 학습이 안되는 문제가 RNN의 고질적인 문제이다.
    - RNN은 activation 함수로 tanh()를 사용한다. tanh()의 gradient는 0 ~ 1 사이의 실수가 나온다. 그래서 sequence가 길어지면 **초기 time step의 값에 대한 weight가 업데이트가 되지 않게** 된다.  
- Time step이 길어지면 초기 Sequence에 정보가 hidden state에서 점점 사라지는 **기억력 소실문제**가 발생한다.
      
![BPTT](figures/rnn/09_bptt.png)

- 이런 Simple RNN의 문제 모델 구조로 해결한 모델이 **LSTM이나 GRU** 모델이다. Sequence 데이터 처리 모델로 이 둘을 주로 사용한다.

# LSTM

## 개요
- Simple RNN을 개선한 변형 알고리즘
    - **오래 기억할 것은 유지하고 잊어버릴 것은 빨리 잊어버리자**
    - 바로 전 time step의 처리 결과와 전체 time step의 처리 결과를 같이 받는다.


![lstm](figures/rnn/09_lstm_layer.png)

- Simple RNN과 다르게 두개의 이전 time step까지의 처리결과를 사용한다.
    - **Cell State**
        - Long term memory 로 전체 step에 대한 누적 기억값
    - **Hidden State**
        - Short term memory 로 이전 sequence 에 대한 기억값

## LSTM  Gate
- Gate
    - LSTM은 Layer내에 여러개의 **Gate 연산** 통해 성능을 높인다.
    - Sigmoid연산을 이용해 값의 얼마를 적용할지 정한다. 이것을 문이 열리고 닫히는 의미로 Gate라고 한다.
- LSTM은 3개 Gate를 사용한다.
    -  **Forget gate**
    -  **Input gate**
    -  **Output gate**
-  각 Gate는 hidden state와 Input data에 곱하는 Weight들을 가진다.

![lstm](figures/rnn/10_lstm_layer_all.png)

### Forget gate
![forget gate](figures/rnn/11_forget_gate.jpg)

- 이전 time step의 hidden state와 현재 time step의 입력데이터를 기준으로 장기 기억인 cell state에서 얼마나 잊을 지 계산한다.

### Input Gate

![input gate](figures/rnn/12_input_gate.jpg)
- 이전 time step의 hidden state와 현재 time step의 입력데이터를 cell state에서 얼마나 반영할 지 계산한다.

### Cell State update

![update](figures/rnn/13_cell_state_update.jpg)
- cell state는 forget gate의 결과를 곱해서 얼마나 잊을지 결정하고 input gate의 결과를 더해서 현재 입력에 대한 정보를 추가한다.

### Output Gate
![output gate](figures/rnn/14_output_gate.jpg)
- 이전 time step의 hidden state와 현재 time step의 입력데이터, 현재 step의 입력이 반영된 Cell state를 이용해 출력 결과를 계산한다.
-  **Output gate의 계산 결과는 다음 step에 hidden state(단기기억)로 전달된다.**

<Image 참조> https://www.pluralsight.com/guides/introduction-to-lstm-units-in-rnn

## Pytorch LSTM layer

- [nn.LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)
    - **파라미터**
        - input_size: 입력 데이터의 shape
        - hidden_size: Layer의 Hidden size
        - num_layers: 몇층으로 Layer을 쌓을지 개수
        - batch_first: True - (batch, seqence len, ..) False - (sequence len, batch, ..). Default: False
        - dropout: Dropout rate 비율
        - bidirectional:  양방향 적용 여부. Default: False
     
### 추론시 Input tensor 구조
- Input_data, (Hidden_state, Cell_state)
- Input_data의 shape
    - **(sequence_length, batch_size, input_feature_shape)**
- (Hidden_state, Cell_state)는 생략시 0 입력된다.
    - Hidden_state의 shape
        - **(D * layer수, batch_size, hidden_size)**
        - D: 양방향(bidirectional) 이면 2 아니면 1
    - Cell_state의 shape
        - **(D * layer수, batch_size, hidden_size)**
        - D: 양방향(bidirectional) 이면 2 아니면 1
### Output tentor 구조
- Output_data, (Hidden_state, Cell_state)
  
#### Output_data의 shape
- 모든 timestep의 출력결과를 묶어서 반환
- **(sequence length, batch_size, D * hidden_size)**
#### Hidden_state의 shape
- 마지막 timestep의 출력결과
- **(D * layer수, batch_size, hidden_size)**
- D: 양방향(bidirectional) 이면 2 아니면 1
#### Cell_state의 shape
- cell state(장기기억) 값
- **(D * layer수, batch_size, hidden_size)**
- D: 양방향(bidirectional) 이면 2 아니면 1